In [1]:
from sys import modules

IN_COLAB = 'google.colab' in modules
if IN_COLAB:
    !pip install -q ir_axioms[examples] python-terrier

In [2]:
# Start/initialize PyTerrier.
from pyterrier import started, init

if not started():
    init(tqdm="auto")

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
from pyterrier.datasets import get_dataset, Dataset

# Load dataset.
dataset_name = "antique"
dataset: Dataset = get_dataset(f"irds:{dataset_name}")
dataset_train: Dataset = get_dataset(f"irds:{dataset_name}/train")
dataset_test: Dataset = get_dataset(f"irds:{dataset_name}/test")

In [4]:
from pathlib import Path

cache_dir = Path("cache/")
index_dir = cache_dir / "indices" / dataset_name.split("/")[0]

In [5]:
from pyterrier.index import IterDictIndexer

if not index_dir.exists():
    indexer = IterDictIndexer(str(index_dir.absolute()))
    indexer.index(
        dataset.get_corpus_iter(),
        fields=["text"]
    )

In [6]:
from pyterrier.batchretrieve import BatchRetrieve

# BM25 baseline retrieval.
bm25 = BatchRetrieve(str(index_dir.absolute()), wmodel="BM25", num_results=20)

In [8]:
from ir_axioms.axiom import (
    ArgUC, QTArg, QTPArg, aSL, PROX1, PROX2, PROX3, PROX4, PROX5, TFC1, TFC3, RS_TF, RS_TF_IDF, RS_BM25, RS_PL2, RS_QL,
    AND, LEN_AND, M_AND, LEN_M_AND, DIV, LEN_DIV, M_TDC, LEN_M_TDC, STMC1, STMC1_f, STMC2, STMC2_f, LNC1, TF_LNC, LB1,
    REG, ANTI_REG, REG_f, ANTI_REG_f, ASPECT_REG, ASPECT_REG_f, ORIG
)

axioms = [
    # ~ArgUC(), ~QTArg(), ~QTPArg(), ~aSL(),
    # ~LNC1(), ~TF_LNC(), ~LB1(),
    ~PROX1(), ~PROX2(), ~PROX3(), ~PROX4(), ~PROX5(),
    # ~REG(), ~REG_f(), ~ANTI_REG(), ~ANTI_REG_f(), ~ASPECT_REG(), ~ASPECT_REG_f(),
    # ~AND(), ~LEN_AND(), ~M_AND(), ~LEN_M_AND(), ~DIV(), ~LEN_DIV(),
    # ~RS_TF(), ~RS_TF_IDF(), ~RS_BM25(), ~RS_PL2(), ~RS_QL(),
    # ~TFC1(), ~TFC3(), ~M_TDC(), ~LEN_M_TDC(),
    ~STMC1(), ~STMC1_f(), ~STMC2(), ~STMC2_f(),
    ORIG()
]

In [9]:
from statistics import mean
from ir_axioms.backend.pyterrier.transformers import AggregatedAxiomaticPreference

features = bm25 % 10 >> AggregatedAxiomaticPreference(
    axioms=axioms,
    index=index_dir,
    aggregations=[mean, min, max],
    dataset=dataset_name,
    verbose=True
)

In [10]:
features.transform(dataset_test.get_topics().iloc[:1])["features"].iloc[0].tolist()

Aggregating query axiom preferences:   0%|          | 0/1 [00:00<?, ?query/s]

[-0.4,
 -1.0,
 1.0,
 -0.4,
 -1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.1,
 -1.0,
 1.0,
 0.1,
 -1.0,
 1.0,
 0.0,
 -1.0,
 1.0,
 0.1,
 0.0,
 1.0,
 0.9,
 0.0,
 1.0]

In [11]:
from pyterrier.ltr import apply_learned_model
from sklearn.ensemble import RandomForestRegressor

random_forest = RandomForestRegressor(n_estimators=400)
ltr = features >> apply_learned_model(random_forest)

In [12]:
ltr.fit(dataset_train.get_topics(), dataset_train.get_qrels())

Aggregating query axiom preferences:   0%|          | 0/2413 [00:00<?, ?query/s]

KeyboardInterrupt: 

In [ ]:
from pyterrier.pipelines import Experiment
from ir_measures import nDCG, MAP

experiment = Experiment(
    [bm25, ltr ^ bm25],
    dataset_test.get_topics(),
    dataset_test.get_qrels(),
    [nDCG @ 5, nDCG @ 10, nDCG @ 20, MAP],
    ["BM25", "KwikSort Random Forest"],
    verbose=True,
)
experiment.sort_values(by="nDCG@10", ascending=False, inplace=True)

In [ ]:
experiment